In [ ]:
# !pip install pretty_midi

## Convert MIDI files

In [ ]:
import pretty_midi
import numpy as np
import glob
import os

In [ ]:
from zipfile import ZipFile

file_name = "converter.zip"
with ZipFile(file_name,'r') as myzip:
    ZipFile.extractall(myzip)

In [ ]:
def strech_midi(path):
    midi = pretty_midi.PrettyMIDI(path)
    e_time = midi.get_end_time()
    r_bpm = 60/e_time*32
    tolerance = 0.01
    
    if abs(np.round(r_bpm)-r_bpm) > tolerance:
        t = int(r_bpm)
        m = np.asarray([60/((t+i)*4) for i in range(1,5)])
        l = np.outer(m,np.arange(1,5)).flatten()
        tmp_bpms = np.asarray([60/(e_time-i)*32 for i in l])
        diff = abs(np.round(tmp_bpms)-tmp_bpms).reshape((4,4))
        ind = np.unravel_index(np.argmin(diff, axis=None), diff.shape)
        r_bpm = t+ind[0]+1

        print(path,r_bpm,ind[0]+1)

    midi_new = pretty_midi.PrettyMIDI(resolution=240,initial_tempo=np.round(r_bpm))

    program = pretty_midi.instrument_name_to_program('Acoustic Grand Piano')
    piano = pretty_midi.Instrument(program=program)

    nt_list = midi.instruments[0].notes
    for note in nt_list:
        piano.notes.append(note)

    midi_new.instruments.append(piano)
    return midi_new

In [ ]:
def midi_converter(path):
    file_path = glob.glob(path)
    file_path.sort(key= str.lower)
    for f in file_path:
        name = f.split('/')[-1]
        midi_new = strech_midi(f)
        midi_new.write('midinew/{}'.format(name))


In [ ]:
midi_converter("converter/*.mid")

In [ ]:
def get_all_file_paths(directory): 
    file_paths = [] 
    for root, directories, files in os.walk(directory): 
        for filename in files: 
            filepath = os.path.join(root, filename) 
            file_paths.append(filepath) 
    return file_paths         
  
directory = './midinew' 
fp = get_all_file_paths(directory) 

with ZipFile('midinew.zip','w') as zip: 
    for f in fp: 
        zip.write(f) 